## NB2. ROF monthly, annual, seasonal discharge at ocean outlets <a id='top'></a>

Use 

1. reach-D19 gauge link ascii
2. D19 flow site geopackage
3. D19 discharge netCDF
4. monthly and yearly flow netCD (history file)

[1. Setupt](#setup)

[2. Loading data](#load_data)

- monthly history files (directory from CESM or postprocessed) from archive. 

- Reference data is monthly discharge estimates at 922 big river mouths from Dai et al. 2019 data (D19)

[3. Read river, catchment, gauge information](#read_meta)

- catchment polygon (geopackage)
- gauge point (geopackage)
- gauge-catchment link (csv)
- outlet reach information (netCDF) including discharging ocean names

[4. Ocean discharge line plots](#922_rivers)

- total seasonal flow for oceans. 


In [ ]:
%matplotlib inline

import os, sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import cartopy.feature as cfeature
from dask.distributed import Client, LocalCluster

from scripts.utility import load_yaml
from scripts.utility import no_time_variable
from scripts.utility import read_shps
from scripts.utility import get_index_array

rivers_50m = cfeature.NaturalEarthFeature("physical", "rivers_lake_centerlines", "50m")
land = cfeature.LAND

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])
print(xr.__name__, xr.__version__)
print(pd.__name__, pd.__version__)
print(gpd.__name__, gpd.__version__)

-------------------------
## 1. Setup <a id='setup'></a>

In [ ]:
# Parameter Defaults
# parameters are set in CUPiD's config.yml file

CESM_output_dir = ""
case_name = None  # case name
base_case_name = None  # base case name
start_date = ""
end_date = ""
serial = False  # use dask LocalCluster
lc_kwargs = {}

analysis_name = "test"  # Used for Figure png names
rof_start_date = start_date  # specify if different starting yyyy-mm-dd is desired
rof_end_date = end_date  # specify if different ending yyyy-mm-dd is desired
grid_name = "f09_f09_mosart"  # ROF grid name used in case
base_grid_name = grid_name  # spcify ROF grid name for base_case if different than case
figureSave = False

-------------------------
ROF ancillary data specification 

In [ ]:
setup = load_yaml("./setup/setup.yaml")

domain_dir = setup[
    "ancillary_dir"
]  # ancillary directory including such as ROF domain, river network data
geospatial_dir = setup["geospatial_dir"]  # including shapefiles or geopackages
ref_flow_dir = setup["ref_flow_dir"]  # including observed or reference flow data
case_meta = setup["case_meta"]  # RO grid meta
catch_gpkg = setup["catch_gpkg"]  # catchment geopackage meta
reach_gpkg = setup["reach_gpkg"]  # reach geopackage meta
network_nc = setup["river_network"]  # river network meta

In [ ]:
oceans_list = [
    "arctic",
    "atlantic",
    "indian",
    "mediterranean",
    "pacific",
    "south_china",
    "global",
]
case_list = [case for case in [case_name, base_case_name] if case is not None]
grid_list = [grid for grid in [grid_name, base_grid_name] if grid is not None]
time_period = slice(f"{rof_start_date}", f"{rof_end_date}")  # analysis time period
nyrs = int(rof_end_date[:4]) - int(rof_start_date[:4]) + 1  # number of years
nmons = nyrs * 12  # number of months
year_list = [
    "{:04d}".format(yr)
    for yr in np.arange(int(rof_start_date[0:4]), int(rof_end_date[0:4]) + 1)
]

-----
### dasks (optional)

In [ ]:
# Spin up cluster (if running in parallel)
client = None
if serial:
    cluster = LocalCluster(**lc_kwargs)
    client = Client(cluster)

client

## 2. Loading data <a id='load_data'></a>

### 2.1. Monthly/annual flow netCDFs
- month_data (xr dataset)
- year_data (xr dataset)
- seas_data (xr dataset)

In [ ]:
%%time

reachID = {}
month_data = {}
year_data = {}
seas_data = {}
for case, grid in zip(case_list, grid_list):
    in_dire = os.path.join(CESM_output_dir, case, "rof/hist")
    model = case_meta[grid]["model"]
    domain = case_meta[grid]["domain_nc"]
    var_list = case_meta[grid]["vars_read"]

    def preprocess(ds):
        return ds[var_list]

    # monthly
    nc_list = []
    for nc_path in sorted(glob.glob(f"{in_dire}/{case}.{model}.h*.????-*.nc")):
        for yr in year_list:
            if yr in os.path.basename(nc_path):
                nc_list.append(nc_path)
    month_data[case] = (
        xr.open_mfdataset(
            nc_list,
            data_vars="minimal",
            parallel=True,
            preprocess=preprocess,
        )
        .sel(time=time_period)
        .load()
    )
    # annual
    year_data[case] = month_data[case].resample(time="YS").mean(dim="time")

    # seasonal (compute here instead of reading for conisistent analysis period)
    seas_data[case] = month_data[case].groupby("time.month").mean("time")
    vars_no_time = no_time_variable(month_data[case])
    if vars_no_time:
        seas_data[case][vars_no_time] = seas_data[case][vars_no_time].isel(
            month=0, drop=True
        )
    time = month_data[case]["time"]
    if domain == "None":
        reachID[case] = month_data[case]["reachID"].values
    else:
        reachID[case] = (
            xr.open_dataset(f"{domain_dir}/{domain}")["reachID"]
            .stack(seg=("lat", "lon"))
            .values
        )
    print(case)

### 2.2 D19 discharge data
- ds_q_obs_mon (xr datasets)
- ds_q_obs_yr (xr datasets)
- dr_q_obs_seasonal (xr datasets)

In [ ]:
%%time

# read monthly data
ds_q = xr.open_dataset(
    "%s/D09/coastal-stns-Vol-monthly.updated-May2019.mod.nc" % (ref_flow_dir),
    decode_times=False,
)
ds_q["time"] = xr.cftime_range(
    start="1900-01-01", end="2018-12-01", freq="MS", calendar="standard"
)

# monthly
obs_available = True
if ds_q["time"].sel(time=time_period).values.size == 0:
    obs_available = False
    ds_q_obs_mon = xr.DataArray(
        data=np.ones((len(time), len(ds_q["station"])), dtype="float") * np.nan,
        dims=["time", "station"],
        coords=dict(
            station=ds_q["station"],
            time=time,
        ),
    )
else:
    ds_q_obs_mon = ds_q["FLOW"].sel(time=time_period)
# compute annual flow from monthly
ds_q_obs_yr = ds_q_obs_mon.resample(time="YE").mean(dim="time")
# compute annual cycle at monthly scale
dr_q_obs_seasonal = ds_q_obs_mon.groupby("time.month").mean("time")

## 3. Reading river, catchment, gauge infomation  <a id='read_meta'></a>

- catchment polygon (geopackage)
- gauge point (geopackage)
- gauge-catchment link (csv)
- outlet reach information (netCDF)

### 3.1. reach-D19 gauge link csv
- gauge_reach_lnk (dataframe)

In [ ]:
gauge_reach_lnk = {}
for case, grid in zip(case_list, grid_list):
    gauge_reach_lnk[case] = pd.read_csv(
        "%s/D09/D09_925.%s.asc" % (ref_flow_dir, case_meta[grid]["network"])
    )

### 3.2 D19 flow site shapefile
- gauge_shp (dataframe)

In [ ]:
%%time

gauge_shp = gpd.read_file(
    os.path.join(ref_flow_dir, "D09", "geospatial", "D09_925.gpkg")
)
gauge_shp = gauge_shp[gauge_shp["id"] != 9999999]

In [ ]:
%%time

ocean_shp = gpd.read_file(os.path.join(geospatial_dir, "oceans.gpkg"))

### 3.3 Read river network information
- riv_ocean (dataframe)

In [ ]:
%%time

## read catchment geopackage
gdf_cat = {}
for case, grid in zip(case_list, grid_list):
    cat_gpkg = os.path.join(
        geospatial_dir, catch_gpkg[grid]["file_name"]
    )  # geopackage name
    id_name_cat = catch_gpkg[grid]["id_name"]  # reach ID in geopackage
    var_list = [id_name_cat]
    if "lk" in grid_name:
        var_list.append("lake")
    gdf_cat[case] = read_shps([cat_gpkg], var_list)

In [ ]:
%%time

# read river outlet netcdf
riv_ocean = {}
for case, grid in zip(case_list, grid_list):
    riv_ocean_file = os.path.join(
        domain_dir, network_nc[grid]["file_name"].replace(".aug.nc", ".outlet.nc")
    )  # network netcdf name
    ds_rn_ocean = xr.open_dataset(riv_ocean_file).set_index(seg="seg_id")
    df_tmp = ds_rn_ocean.to_dataframe()
    riv_ocean[case] = pd.merge(
        gdf_cat[case], df_tmp, left_on=catch_gpkg[grid]["id_name"], right_index=True
    )

### 2.6 Merge gauge, outlet catchment dataframe

- gauge_shp1 (dataframe)

In [ ]:
%%time

# Merge gauge_reach lnk (dataframe) into gauge shapefile
gauge_shp1 = {}
for case, grid in zip(case_list, grid_list):
    df = gauge_reach_lnk[case]

    # df = df.loc[(df['flag'] == 0)]
    df1 = df.drop(columns=["riv_name"])
    df2 = pd.merge(gauge_shp, df1, how="inner", left_on="id", right_on="gauge_id")
    gauge_shp1[case] = pd.merge(
        df2,
        riv_ocean[case],
        how="inner",
        left_on="route_id",
        right_on=catch_gpkg[grid]["id_name"],
    )

------
## 4. plot annual cycle for global oceans <a id='24_large_rivers'></a>

TODO: Referece flow plot should be independent from cases (network). Currently the last case plotted looks better matched with reference flow. 

In [ ]:
%time

nrows = 4
ncols = 2
fig, axes = plt.subplots(nrows, ncols, figsize=(7.25, 6.5))
plt.subplots_adjust(
    top=0.95, bottom=0.065, right=0.98, left=0.10, hspace=0.225, wspace=0.250
)  # create some space below the plots by increasing the bottom-value

for ix, ocean_name in enumerate(oceans_list):
    row = ix // 2
    col = ix % 2
    for case, grid in zip(case_list, grid_list):

        q_name = case_meta[grid]["flow_name"]

        if case_meta[grid]["network_type"] == "vector":
            if ocean_name == "global":
                id_list = gauge_shp1[case]["route_id"].values
            else:
                id_list = gauge_shp1[case][gauge_shp1[case]["ocean"] == ocean_name][
                    "route_id"
                ].values
            reach_index = get_index_array(reachID[case], id_list)
            dr_flow = seas_data[case][q_name].isel(seg=reach_index).sum(dim="seg")
            dr_flow.plot(ax=axes[row, col], linestyle="-", lw=0.75, label=case)

        elif case_meta[grid_name]["network_type"] == "grid":  # means 2d grid
            if ocean_name == "global":
                id_list = gauge_shp1[case]["route_id"].values
            else:
                id_list = gauge_shp1[case][gauge_shp1[case]["ocean"] == ocean_name][
                    "route_id"
                ].values

            reach_index = get_index_array(reachID[case], id_list)
            seas_data_vector = seas_data[case][q_name].stack(seg=("lat", "lon"))
            dr_flow = seas_data_vector.isel(seg=reach_index).sum(dim="seg")
            dr_flow.plot(ax=axes[row, col], linestyle="-", lw=0.75, label=case)

    # reference data
    if obs_available:
        if ocean_name == "global":
            id_list = gauge_shp1[case]["id"].values
        else:
            id_list = gauge_shp1[case][gauge_shp1[case]["ocean"] == ocean_name][
                "id"
            ].values
        gauge_index = get_index_array(ds_q["id"].values, id_list)
        dr_obs = dr_q_obs_seasonal.isel(station=gauge_index).sum(dim="station")
        dr_obs.plot(
            ax=axes[row, col],
            linestyle="None",
            marker="o",
            markersize=2,
            c="k",
            label="D19",
        )

    axes[row, col].set_title("%d %s" % (ix + 1, ocean_name), fontsize=9)
    axes[row, col].set_xlabel("")
    if row < 7:
        axes[row, col].set_xticklabels("")
    if col == 0:
        axes[row, col].set_ylabel("Mon. flow [m$^3$/s]", fontsize=9)
    else:
        axes[row, col].set_ylabel("")
    axes[row, col].tick_params("both", labelsize="x-small")

# Legend- make space below the plot-raise bottom. there will be an label below the second last (bottom middle) ax, thanks to the bbox_to_anchor=(x, y) with a negative y-value.
axes[row, col].legend(
    loc="center left", bbox_to_anchor=(1.10, 0.40, 0.75, 0.1), ncol=1, fontsize="small"
)

for jx in range(ix + 1, nrows * ncols):
    row = jx // 2
    col = jx % 2
    fig.delaxes(axes[row][col])

if figureSave:
    plt.savefig(f"./NB2_Fig1_ocean_discharge_season_{analysis_name}.png", dpi=200)